In [1]:
from pathlib import Path
import sys

sys.path.append('/home/ovoinov/work/sk3d/dev.sk_robot_rgbd_data/src')
from skrgbd.data.dataset.scene_paths import ScenePaths
from skrgbd.data.dataset.params import stl_light_setups, stl_view_ids, stl_val_view_ids
from skrgbd.utils.logging import tqdm
from skrgbd.data.dataset.dataset import wip_scene_id_by_name

In [ ]:
scene_name = 'dragon'
img_var = 'raw'
scene_paths = ScenePaths(scene_name, data_dir='/mnt/datasets/sk3d/dataset',
                         raw_dir='/mnt/datasets/sk3d/raw')

lines = []
for (scan_var, mode, view_ids) in [('ref', 'partial', stl_view_ids),
                                   ('val', 'validation', stl_val_view_ids)]:
    for cam_name in ['stl_right', 'stl_left']:
        for light in stl_light_setups:
            dst_dir = Path(scene_paths.img(cam_name, mode, 0, light, img_var)).parent
            lines.append(f'mkdir -p {dst_dir}\n')
            for view_i in view_ids:
                src_path = scene_paths.sl_img(cam_name, view_i, light, scan_var)
                dst_path = scene_paths.img(cam_name, mode, view_i, light, img_var)
                lines.append(f'ln {src_path} {dst_path}\n')
    lines.append(f'if [ ! -d {scene_paths.sl_raw("val")} ]; then exit; fi\n')
    
with open('/home/ovoinov/Downloads/link_imgs.sh', 'w') as file:
    file.write(''.join(lines).replace('/mnt/datasets/sk3d', '/mnt/backups/sk3d').replace(scene_name, "${1}"))